In [0]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
 
spark = SparkSession.builder \
    .appName("IngestaoBronze") \
    .getOrCreate()

In [0]:
%run ./Intancia_Containers

In [0]:
import zipfile
import os

def unzip_file(path_origem, container_nome):
    """
    Copia o ZIP do Blob Storage para o DBFS local e extrai os CSVs.
    """

    # Nome do arquivo
    zip_name = os.path.basename(path_origem)

    # Caminho temporário no DBFS
    dbfs_zip_path = f"dbfs:/tmp/{zip_name}"
    local_zip_path = f"/dbfs/tmp/{zip_name}"

    # Copia do Blob (wasbs) para DBFS
    dbutils.fs.cp(path_origem, dbfs_zip_path, recurse=False)

    # Pasta de extração
    extract_path = f"/dbfs/tmp/unzipped/{container_nome}"
    os.makedirs(extract_path, exist_ok=True)

    # Extrai os arquivos
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

        extracted_files = [
            f"/dbfs/tmp/unzipped/{container_nome}/{f}"
            for f in zip_ref.namelist()
        ]

    # Retorna caminhos compatíveis com Spark
    return [f.replace("/dbfs", "dbfs:") for f in extracted_files]

In [0]:
%run ./Colunas_CNPJ

In [0]:
def renomeia_colunas(df, nome_tabela):
    # Normaliza o nome (remove números como Empresas0, Empresas1)
    nome_normalizado = ''.join(filter(str.isalpha, nome_tabela))

    colunas_corretas = colunas_map.get(nome_normalizado)

    if not colunas_corretas:
        print(f"[Aviso] Nenhum dicionário encontrado para '{nome_tabela}'.")
        return df

    if len(df.columns) != len(colunas_corretas):
        print(
            f"[Aviso] '{nome_tabela}': número de colunas diferente "
            f"(DF={len(df.columns)} | Esperado={len(colunas_corretas)})."
        )
        return df

    # Renomeia todas as colunas de uma vez (mais performático)
    return df.toDF(*colunas_corretas)

In [0]:
TGT_STORAGE_ACCOUNT = dbutils.secrets.get(scope="secrets-kv", key="tgt-storage-account")
TGT_CONTAINER = dbutils.secrets.get(scope="secrets-kv", key="tgt-container")
TGT_SAS_TOKEN = dbutils.secrets.get(scope="secrets-kv", key="tgt-sas-token")

# Configuring SAS for Lakehouse Container
spark.conf.set(
    f"fs.azure.sas.{TGT_CONTAINER}.{TGT_STORAGE_ACCOUNT}.blob.core.windows.net",
    TGT_SAS_TOKEN
)

LAKEHOUSE_BASE_PATH = f"wasbs://{TGT_CONTAINER}@{TGT_STORAGE_ACCOUNT}.blob.core.windows.net/bronze_1"

In [0]:
from pyspark.sql.functions import max as spark_max, col

ultima_atualizacao_bronze = (
    spark.table("bronze_1.bronze_control_table")
         .filter(col("container_name") == "CNPJ")
         .select(spark_max("last_ingestion_timestamp").alias("ultima_atualizacao"))
         .collect()[0]["ultima_atualizacao"]
)

print(ultima_atualizacao_bronze)

df_inventario = (
    df_inventario
        .filter(col("container_name") == "CNPJ")
        if ultima_atualizacao_bronze is None
        else df_inventario
            .filter(col("container_name") == "CNPJ")
            .filter(col("modificationTime") > ultima_atualizacao_bronze)
)

display(df_inventario)

In [0]:
import re

tabelas_bronze = {}
controle_ingestao = {}

# iterando nos Arquivos
for row in df_inventario.collect():
    path_origem = row['path']
    container = row['container_name']
    nome_arquivo = row['name']
    
    # Removendo a extensão para o nome da tabela Delta
    tabela_nome = nome_arquivo.split('.')[0].lower()
    print(f"⏳ Processando: {tabela_nome}")

    try:
        if not nome_arquivo.endswith('.zip'):
            print(f"❌ Arquivo {nome_arquivo} não é ZIP")
            continue

        print(f"📦 Descompactando: {nome_arquivo}")
        arquivos_extraidos = unzip_file(path_origem, container)

        if not arquivos_extraidos:
            print(f"⚠️ Nenhum CSV encontrado em {nome_arquivo}")
            continue

        # Remove sufixo numérico → empresas0 → empresas
        nome_tabela = re.sub(r"\d+$", "", tabela_nome)

        for arquivo in arquivos_extraidos:
            df_temp = spark.read.format("csv") \
                .option("header", "false") \
                .option("sep", ";") \
                .option("encoding", "UTF-8") \
                .load(arquivo)
            
            df_temp = renomeia_colunas(df_temp, nome_tabela)

            df_temp = df_temp.select(
                [col(c).cast("string").alias(c) for c in df_temp.columns]
            )

            # Agrupa DataFrames por tabela lógica
            tabelas_bronze.setdefault(nome_tabela, []).append(df_temp)

            # Controle de ingestão
            controle_ingestao.setdefault(nome_tabela, set()).add(path_origem)

    except Exception as e:
        print(f"❌ Erro ao processar {nome_arquivo}: {e}")

In [0]:
from functools import reduce

tabelas_unificadas = {}

for nome_tabela, lista_dfs in tabelas_bronze.items():
    df_final = reduce(
        lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True),
        lista_dfs
    )
    tabelas_unificadas[nome_tabela] = df_final

In [0]:
from pyspark.sql.functions import current_timestamp, current_date

TABELAS_SNAPSHOT = {"empresas", "estabelecimentos", "socios", "simples"}

for nome_tabela, df in tabelas_unificadas.items():
    destino = f"{LAKEHOUSE_BASE_PATH}/{container.lower()}/{nome_tabela}"

    print(f"✅ Gravando tabela bronze: {nome_tabela}")

    df = (
        df
        .withColumn("_ingestion_date", current_date())
    )
    
    # Snapshot somente para tabelas CNPJ principais
    if nome_tabela in TABELAS_SNAPSHOT:
        df.write \
            .format("delta") \
            .mode("append") \
            .partitionBy("_ingestion_date") \
            .save(destino)

    else:
        # ✅ Demais tabelas seguem ingestão padrão
        df.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .save(destino)

    controle_df = spark.createDataFrame(
        [("CNPJ", nome_tabela, f"{controle_ingestao.get(nome_tabela, [])}")],
        ["container_name", "table_name", "input_file_name"]
    ).withColumn(
        "last_ingestion_timestamp", current_timestamp()
    )

    controle_df.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable("bronze_1.bronze_control_table")
